# Tools

In [2]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd

pd.options.display.max_columns=None
pd.set_option('max_colwidth', None)  # 設定 value 顯示長度

# 開爬
> **爬取每日熱門爬取每日熱門文章。**

> **爬取資訊**
    >- 看板
    >- 文章標題
    >- 作者
    >- 發文時間
    >- 文章連結
    >- 推文數
    >- 文章內容
    >- 推文內容

> **資訊位置**
    >- 'div', 'r-ent'
    - 文章標題

>- 流程
    1. 輸入URL，獲取網頁原始碼
    2. 透過 BeautifulSoup 解析 HTML 格式文件
    3. 從最新頁面開始爬取
        - 設定爬取條件
        - 將需求資訊以 dict 儲存
        - return 上一頁 URL
    4. 往上一頁爬取（設定爬取條件）

In [3]:
url = "https://www.ptt.cc/bbs/NBA/index.html"

### 1. 取得 response
r = requests.get(url)  # cookies={'over18': '1'}

### 2. 透過 parser 轉換成 BeautifulSoup 比較好處理的格式（以 BeautifulSoup 解析 HTML 格式文件）
"""
    - soup是獲得的文件物件, html.parser是支援的parser.
    - 關於parser, 比較常見的parser有這些：html.parser / html5lib / lxml。
    
    - BeautifulSoup 會根據指定的 parser 把文件轉換成一個複雜的樹形結構，每個節點都是 Python 物件。
        - 所有物件大致上可歸類為4種:
            - BeautifulSoup: 表示一個文件的所有內容.
            - Tag: 基本上就是HTML裡的各種tag(div, p, etc...). 每個tag有兩個重要的屬性: 
                - name: 即tag的名字或著tag本身的name.
                - attrs: 通常指一個tag的class(常用).
            - NavigableString: 取得tag內的文字, 如: soup.p.string
            - Comment: 此物件是一個特殊的NavigableString物件, 其輸出的內容不包含註解符號.
"""
soup = BeautifulSoup(r.text, "html.parser")

### 3. Results
print('status_code: ', r.status_code)
# print('未解析資料: ', r.text)
print('解析後資料', soup)

status_code:  200
解析後資料 <!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>看板 NBA 文章列表 - 批踢踢實業坊</title>
<link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/NBA/index.html"><span class="board-label">看板 </span>NBA</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</di

## 文章標題

In [4]:
# 尋找 class="title" 的 div 標籤
titles = soup.find('div', class_='title')
print('標題文字：', titles.a.string)

標題文字： [新聞] 衛少陷生涯最大低潮　帕金斯：信心已經


In [5]:
# 尋找所有 class="title" 的 div 標籤
titles = soup.find_all('div', class_='title')
for title in titles:
    print(title.a.string)

[新聞] 衛少陷生涯最大低潮　帕金斯：信心已經
[花邊] 教練評價不夠性感，Jokic：他沒有說謊
[新聞] 讚英格倫進攻高水準 鵜鶘主帥:天空才是極
[公告] 板規v9.0
[情報] SEASON Schedule January 21–22
[情報] NBA Standings (Jan. 14, 2022)


## test

In [6]:
# 爬取整篇文章
articles = soup.find_all('div', 'r-ent')

for article in articles:
    PTT_URL = "https://www.ptt.cc"
    page_div = soup.find('div', 'btn-group btn-group-paging')
    prev_page = page_div.find_all('a')[1]['href']  # 上頁
    latest_page = page_div.find_all('a')[3]['href']  # 最新
    
    today = dt.datetime.today().date().strftime('%m/%d').lstrip('0')
    date = article.find('div', 'date').text.strip()  # strip(): 去除字串頭尾的空格
    
    if date == today:
        if article.find('div', 'title').a == None:
            print(None)
        else:
            print(article.find('div', 'title').a.text)
    
    r = requests.get(PTT_URL + prev_page)
    soup = BeautifulSoup(r.text, "html.parser")

[新聞] 衛少陷生涯最大低潮　帕金斯：信心已經
[花邊] 教練評價不夠性感，Jokic：他沒有說謊
[新聞] 讚英格倫進攻高水準 鵜鶘主帥:天空才是極
[情報] NBA Standings (Jan. 14, 2022)


In [7]:
single_article = soup.find_all('div', 'r-ent')[2]

# 需求資訊
title = single_article.find('div', 'title').a.text  # 標題
push_count = single_article.find('div', 'nrec').text  # 推文數 (str)
author = single_article.find('div', 'author').text  # author
date = single_article.find('div', 'date').text  # Date (str)
href = single_article.find('a')['href']  # URL = https://www.ptt.cc + href
board = soup.find('a', 'board').text.lstrip('看板 ')

# 頁面操作
page_div = soup.find('div', 'btn-group btn-group-paging')
prev_page = page_div.find_all('a')[1]['href']  # 上頁
latest_page = page_div.find_all('a')[3]['href']  # 最新
board

'NBA'

In [8]:
date = single_article.find('div', 'date').text
dt.datetime.today().date().strftime('%m/%d').lstrip('0') == date.strip()
dt.datetime.today().date().strftime('%m/%d').lstrip('0')

'1/14'

## modules
> **爬取資訊**
    >- 看板
    >- 文章標題
    >- 作者
    >- 發文時間
    >- 文章連結
    >- 推文數
    >- 文章內容
    >- 推文內容

In [9]:
def get_webPage(url):
    r = requests.get(url=url, cookies={'over18': '1'})  # cookies：有些看板會擋是否已滿18歲
    if r.status_code != 200:
        print('URL 無效，status code = ', r.status_code)
    else:
        return r.text
    
def get_articles(page):
    # parse
    soup = BeautifulSoup(page, "html.parser")
    divs = soup.find_all('div', 'r-ent')
    
    # 頁面資訊
    PTT_URL = "https://www.ptt.cc"
    board = soup.find('a', 'board').text.lstrip('看板 ')
    page_div = soup.find('div', 'btn-group btn-group-paging')
    prev_page_url = PTT_URL + page_div.find_all('a')[1]['href']  # 上頁
#     latest_page_url = PTT_URL + page_div.find_all('a')[3]['href']  # 最新
    
    # 日期
    today = dt.datetime.today().date().strftime('%m/%d').lstrip('0')
    
    # 需求欄位
    titles=[]
    push_counts=[]
    authors=[]
    hrefs=[]
    
    # 最新頁面
    for article in divs:
        date = article.find('div', 'date').text.strip()  # strip(): 去除字串頭尾的空格
        
        # 篩選當日文章
        if date == today:
            # 排除已刪除貼文
            if article.find('div', 'title').a == None:
                continue
            else:
                # 需求資訊
                title = article.find('div', 'title').a.text  # 標題
                push_count = article.find('div', 'nrec').text  # 推文數 (str)
                author = article.find('div', 'author').text  # author
                href = PTT_URL + article.find('a')['href']  # URL = https://www.ptt.cc + href
                
                titles.append(title)
                push_counts.append(push_count)
                authors.append(author)
                hrefs.append(href)
        else:
            continue
    
    # 前一頁
    prev_page = get_webPage(prev_page_url)
        
    data = {'date': today, 
            'board': board, 
            'title': titles, 
            'push_count': push_counts, 
            'author': authors, 
            'href': hrefs}
    
    return data, prev_page_url

## 主程式
- 再包一層：get_ptt_data()

In [10]:
all_data = []
page = get_webPage("https://www.ptt.cc/bbs/NBA/index.html")
dict_data, prev_page_url = get_articles(page)
all_data.append(dict_data)

# 往前 5 頁
count = 0
while count < 5:
    page = get_webPage(prev_page_url)
    dict_data, prev_page_url = get_articles(page)
    all_data.append(dict_data)
    count += 1

df = pd.DataFrame()
for data in all_data:
    df = pd.concat([df, pd.DataFrame(data)])

In [11]:
df

,date,board,title,push_count,author,href
0,1/14,NBA,[新聞] 衛少陷生涯最大低潮 帕金斯：信心已經,72,arod1414,https://www.ptt.cc/bbs/NBA/M.1642142004.A.A9E.html
1,1/14,NBA,[花邊] 教練評價不夠性感，Jokic：他沒有說謊,21,LABOYS,https://www.ptt.cc/bbs/NBA/M.1642145052.A.877.html
2,1/14,NBA,[新聞] 讚英格倫進攻高水準 鵜鶘主帥:天空才是極,17,thnlkj0665,https://www.ptt.cc/bbs/NBA/M.1642145412.A.2C1.html
3,1/14,NBA,"[情報] NBA Standings (Jan. 14, 2022)",13,guardyo,https://www.ptt.cc/bbs/NBA/M.1642138204.A.239.html
0,1/14,NBA,[BOX ] Clippers 89:113 Pelicans 數據,16,Rambo,https://www.ptt.cc/bbs/NBA/M.1642130109.A.040.html
1,1/14,NBA,[BOX ] Timberwolves 108:116 Grizzlies 數據,58,Rambo,https://www.ptt.cc/bbs/NBA/M.1642131342.A.E86.html
2,1/14,NBA,[花邊] Bobby Portis 慶祝動作,52,hijacknokia,https://www.ptt.cc/bbs/NBA/M.1642131497.A.1AF.html
3,1/14,NBA,[BOX ] Thunder 130:109 Nets 數據,78,Rambo,https://www.ptt.cc/bbs/NBA/M.1642132256.A.CE8.html
4,1/14,NBA,[討論] Gobert跟Green的重要性被低估了嗎,51,k7202001,https://www.ptt.cc/bbs/NBA/M.1642134299.A.362.html
5,1/14,NBA,[情報] 湖人本季對戰超過五成勝率的球隊僅五勝,83,arod1414,https://www.ptt.cc/bbs/NBA/M.1642134306.A.410.html


In [12]:
df.shape[0]

43